# HW5
```
author: Zeyuan Pan
```

In [2]:
import pandas as pd
import numpy as np
from gurobipy import *

df=pd.read_excel("./DataforHomework5.xlsx")
df.head()

,Year,Stock,Treasury Bond,Money Market,NASDAQ
0,1961,26.81,2.20,2.33,31.664780
1,1962,-8.78,5.72,2.93,-15.024354
2,1963,22.69,1.79,3.38,20.445586
3,1964,16.36,3.71,3.85,23.118500
4,1965,12.36,0.93,4.32,17.152602


In [18]:
#q1.1 mean
asset=["Stock","Treasury Bond","Money Market","NASDAQ"]
for i in asset:
    print(f"asset names:",i,df[i].mean())

asset names: Stock 12.044186046511628
asset names: Treasury Bond 7.792325581395349
asset names: Money Market 6.323023255813954
asset names: NASDAQ 12.899098246658248


In [94]:
#q1.2 covariance matrix
data=df[asset]
# data.head()
data.cov() * (len(data)-1)/len(data)

,Stock,Treasury Bond,Money Market,NASDAQ
Stock,277.316982,37.947286,2.044243,348.843452
Treasury Bond,37.947286,112.124204,-2.391886,-6.347138
Money Market,2.044243,-2.391886,11.540049,-4.290331
NASDAQ,348.843452,-6.347138,-4.290331,634.345309


In [87]:
data1=np.array(data)
data1[0][0]

26.81

### Question 2

the non-linear model could be like the formula below:


In [96]:

np.array(data.cov() * (len(data)-1)/len(data))

array([[277.31698248,  37.94728561,   2.04424316, 348.84345198],
       [ 37.94728561, 112.12420389,  -2.3918861 ,  -6.34713811],
       [  2.04424316,  -2.3918861 ,  11.540049  ,  -4.29033052],
       [348.84345198,  -6.34713811,  -4.29033052, 634.34530917]])

In [97]:

# 定义数据
returns= [0.12044186046511628,  0.07792325581395349, 0.06323023255813954, 0.12899098246658248]  # 预期收益率
covariance_matrix = [[277.31698248,  37.94728561,   2.04424316, 348.84345198],
       [ 37.94728561, 112.12420389,  -2.3918861 ,  -6.34713811],
       [  2.04424316,  -2.3918861 ,  11.540049  ,  -4.29033052],
       [348.84345198,  -6.34713811,  -4.29033052, 634.34530917]]
risk_free_rate = 0.03  # 无风险利率

m = Model()

# Define decision variables
x = m.addVars(4, lb=0, vtype=GRB.CONTINUOUS,name="x")  

# Constraints
temp=0
obj_expr=0
for i in range(4):
    temp+=x[i]*(returns[i]-risk_free_rate)

m.addConstr(temp == 1)

for i in range(4):
    for j in range(4):
        obj_expr += x[i] * covariance_matrix[i][j] * x[j]


m.setObjective(obj_expr, GRB.MINIMIZE)


# Solve
m.optimize()
flag=0
obj_expr1=0
for i in range(4):
   flag+=x[i].x

for i in range(4):
    for j in range(4):
        obj_expr1+= x[i].x * covariance_matrix[i][j] * x[j].x

# Output results
for i in range(4):
    print(f"Asset {i+1} weight: {x[i].x/flag:.3f}")

print(f"Objective Value (Maximized Sharpe Ratio's Numerator): {100/np.sqrt(obj_expr1):.4f}")

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x680d8d91
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-02, 1e-01]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 1 rows, 4 columns, 4 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.51193240e+05 -6.51193240e+05  2.27e+03 1.99e-04  1.08e+06     0s
   1   8.24164158e+04 -1.11923805e+05  4.27e+01 3.74e-06

In [111]:
# data1

In [110]:
m = Model("Portfolio MAD")

y=m.addVars(len(data1),lb=0,vtype=GRB.CONTINUOUS)
z=m.addVars(len(data1),lb=0,vtype=GRB.CONTINUOUS)
x=m.addVars(len(returns),lb=0,ub=1,vtype=GRB.CONTINUOUS)

data1 *=0.01
for t in range(len(data1)):
    expr = quicksum((data1[t][i]-returns[i])*x[i] for i in range(len(returns)))
    m.addConstr(y[t] - z[t] == expr)


m.addConstr(quicksum(returns[i]*x[i] for i in  range(len(returns)))>=0.09)
m.addConstr(quicksum(x[i] for i in  range(len(returns)))==1)

m.setObjective(quicksum(y[t]+z[t] for t in range(len(data1))), GRB.MINIMIZE)

m.optimize()
if m.status == GRB.Status.OPTIMAL:
    for i in range(len(returns)):
        print(f"Asset {i+1} weight: {x[i].x:.3f}")

((data1 - returns)  @ np.array([x[i].x for i in range(len(returns))])).sum()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 45 rows, 90 columns and 266 nonzeros
Model fingerprint: 0xce9f9804
Coefficient statistics:
  Matrix range     [5e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-02, 1e+00]
Presolve removed 7 rows and 14 columns
Presolve time: 0.01s
Presolved: 38 rows, 76 columns, 224 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6393023e-01   1.042060e+00   0.000000e+00      0s
      39    2.6937657e+00   0.000000e+00   0.000000e+00      0s

Solved in 39 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.693765702e+00
Asset 1 weight: 0.334
Asset 2 weight: 0.354
Asset 3 weight: 0.274
Asset 4 weight: 0.037


-2.0816681711721685e-17